In [1]:
import numpy as np
import pandas as pd
import math
import copy

In [2]:
def ginisplit(df, label):
        
        l = len(df)
        #j = len(df[0])
        j=0
        mean = df.mean(axis=0)
        #print(mean)
        gini = []
        observed_t = [[0 for _ in range(2)] for _ in range(2)]
        #print(observed_t)
        gini_index = [0 ,0]
        gini_split = 0
        
        left_idxs, right_idxs,threshold = _split(df)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        for i in left_idxs:
            if(label[i] == 0):
                observed_t[0][0] += 1
            else:
                observed_t[0][1] += 1
        
        for i in right_idxs:
            if(label[i] == 0):
                observed_t[1][0] += 1
            else:
                observed_t[1][1] += 1
        
        #print(observed_t)       
        sum_t = list(np.sum(observed_t, axis = 1))
        if sum_t[0]==0 or sum_t[1]==0:
            print(df)
            #print(df.columns)
        for i in range(2):
             gini_index[i] = 1 - (observed_t[i][0]/sum_t[i])**2 - (observed_t[i][1]/sum_t[i])**2 
        gini_split = sum_t[0]*gini_index[0]/l + sum_t[1]*gini_index[1]/l
        gini.append(gini_split)
        #print(gini)
        return gini_split

In [3]:
def _split(data):
        #mean= np.mean(data.iloc[:,node].to_numpy())
        mean= data.mean(axis=0)
        left=[]
        right=[]
        for row in range(data.shape[0]):
            if data.iloc[row]<mean:
                left.append(row)
            else:
                right.append(row)
        #print('split made')
        return left,right,mean

In [4]:
class Node:

    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


In [9]:
class DecisionTree:

    def __init__(self, min_samples_split=2, max_depth=2, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)

    #def predict(self, X):
        #return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def predict(self,x):
        
        predicted_label=[self._traverse_tree(x.iloc[i],self.root) for i in range(x.shape[0])]
        return predicted_label

    def _grow_tree(self, X, y,depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if (depth >= self.max_depth
                or n_labels == 1
                or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            #print('leaf made')
            return Node(value=leaf_value)

        #feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedily select the best split according to gini
        best_feat = self._best_criteria(X, y )
#          _new = copy.deepcopy( )
#          _new.append(best_feat) 
        
        # grow the children that result from the split
        left_idxs, right_idxs,best_thresh = _split(X.iloc[:,best_feat])
        left = self._grow_tree(X.iloc[left_idxs, :], y.iloc[left_idxs],depth+1)
        right = self._grow_tree(X.iloc[right_idxs, :], y.iloc[right_idxs],depth+1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y):
        best_gini = 999
        split_idx= None
        for feat_idx in range(X.shape[1]):
            X_column = X.iloc[:, feat_idx]
            #print(feat_idx)
            gini = ginisplit(X_column,y.to_numpy())
            #print(gini[0])
            if gini< best_gini: #and gini not in  :
                best_gini = gini
                split_idx = feat_idx
        #print('best gini found at ',split_idx)
        return split_idx 
    
#     def _information_gain(self, y, X_column, split_thresh):
#         # parent loss
#         parent_entropy = entropy(y)

#         # generate split
#         left_idxs, right_idxs = self._split(X_column, split_thresh)

#         if len(left_idxs) == 0 or len(right_idxs) == 0:
#             return 0

#         # compute the weighted avg. of the loss for the children
#         n = len(y)
#         n_l, n_r = len(left_idxs), len(right_idxs)
#         e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
#         child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

#         # information gain is difference in loss before vs. after split
#         ig = parent_entropy - child_entropy
#         return ig
   
    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        #print(x[node.feature])
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        y = list(y)
        s0=0
        s1=0
        for i in range(len(y)):
            if y[i]>0:
                s1+=1
            else:
                s0+=1
        if s0> s1:
            return 0
        else :
            return 1
        
    def print_tree(self,tree=None,indent=" "):
        if not tree:
            tree = self.root
        if tree.value!=None:
            print(tree.value)
        else:
            print("x "+ df.columns[tree.feature],"<=",tree.threshold)
            print("%sleft:" % (indent),end="")
            self.print_tree(tree.left,indent+indent)
            print("%sright:" % (indent),end="")
            self.print_tree(tree.right,indent+indent)

In [31]:
df= pd.read_csv('31.csv',header=None)
for row in range(df.shape[0]):
    if df.iloc[row,-1]>0:
        df.iloc[row,-1]=1
    else:
        df.iloc[row,-1]=0
string = 'feature'
df.columns = [string+str(i) for i in range(df.shape[1])]
#print(df.columns)    
max_depth =int(math.log2(df.shape[1]))
#max_depth =4

In [32]:
from sklearn.model_selection import train_test_split
X= df.iloc[:,:-1]
Y= df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [33]:
classifier= DecisionTree(2,4)
classifier.fit(X_train,y_train)
classifier.print_tree()


x feature7 <= 9.177275891406914
 left:x feature14 <= 0.5447193478003083
  left:x feature14 <= 0.27561422175950034
    left:x feature11 <= 0.24507209554140127
        left:0
        right:0
    right:x feature3 <= 8.753521126760564
        left:0
        right:0
  right:x feature7 <= 3.5367022435199305
    left:x feature7 <= 1.4099526066350712
        left:0
        right:0
    right:x feature1 <= 2.3754249635745506
        left:0
        right:0
 right:x feature7 <= 19.949288256227756
  left:x feature11 <= 0.4298615748322148
    left:x feature3 <= 16.151209677419356
        left:0
        right:0
    right:x feature5 <= 57.7372654155496
        left:0
        right:0
  right:x feature3 <= 35.90765171503958
    left:x feature7 <= 25.207619047619048
        left:0
        right:0
    right:x feature4 <= 136.42703862660943
        left:0
        right:1


In [34]:
Y_pred= classifier.predict(X_test)

In [35]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, Y_pred))

Accuracy: 0.8563695416817033


In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
clf = DecisionTreeClassifier(criterion='gini',splitter='best',max_depth=4,random_state=1)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8567304222302418


In [37]:
from matplotlib import pyplot as plt
from sklearn import tree
text_representation = tree.export_text(clf)
print(text_representation)

|--- feature_7 <= 22.50
|   |--- feature_14 <= 0.14
|   |   |--- feature_10 <= 14.50
|   |   |   |--- feature_6 <= 0.50
|   |   |   |   |--- class: 0
|   |   |   |--- feature_6 >  0.50
|   |   |   |   |--- class: 0
|   |   |--- feature_10 >  14.50
|   |   |   |--- feature_7 <= 0.50
|   |   |   |   |--- class: 0
|   |   |   |--- feature_7 >  0.50
|   |   |   |   |--- class: 0
|   |--- feature_14 >  0.14
|   |   |--- feature_7 <= 8.50
|   |   |   |--- feature_7 <= 2.50
|   |   |   |   |--- class: 0
|   |   |   |--- feature_7 >  2.50
|   |   |   |   |--- class: 0
|   |   |--- feature_7 >  8.50
|   |   |   |--- feature_1 <= 1.50
|   |   |   |   |--- class: 0
|   |   |   |--- feature_1 >  1.50
|   |   |   |   |--- class: 0
|--- feature_7 >  22.50
|   |--- feature_4 <= 130.50
|   |   |--- feature_13 <= 0.86
|   |   |   |--- feature_14 <= 0.48
|   |   |   |   |--- class: 0
|   |   |   |--- feature_14 >  0.48
|   |   |   |   |--- class: 0
|   |   |--- feature_13 >  0.86
|   |   |   |--- featur